# Autoscaling Seldon Deployments


## Prerequisites
 
- The cluster should have `metric-server` running in the `kube-system` namespace
- For Kind install `../../testing/scripts/metrics.yaml` See https://github.com/kubernetes-sigs/kind/issues/398
- For Minikube run:
    
    ```
    minikube addons enable metrics-server
    ```
    

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Setup-Cluster) with [Ambassador Ingress](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Ambassador) and [Install Seldon Core](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Install-Seldon-Core). Instructions [also online](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html).

In [1]:
!kubectl create namespace seldon

Error from server (AlreadyExists): namespaces "seldon" already exists


## Create model with v2beta1 autoscaler

To create a model with an HorizontalPodAutoscaler there are three steps:


  1. Ensure you have a resource request for the metric you want to scale on if it is a standard metric such as cpu or memory, e.g.:
  
```
          resources:
            requests:
              cpu: '0.5'
     
```
     
  1. Add an v2beta1 HPA Spec referring to this Deployment, e.g.:
  
```
    - hpaSpec:
        maxReplicas: 3
        minReplicas: 1
        metrics:
        - resource:
            name: cpu
            targetAverageUtilization: 10
          type: Resource

```

The full SeldonDeployment spec is shown below.

In [5]:
!pygmentize model_with_hpa_v2beta1.yaml

apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: seldon-model
spec:
  name: test-deployment
  predictors:
  - componentSpecs:
    - hpaSpec:
        maxReplicas: 3
        metrics:
        - resource:
            name: cpu
            targetAverageUtilization: 10
          type: Resource
        minReplicas: 1
      spec:
        containers:
        - image: seldonio/mock_classifier:1.19.0-dev
          imagePullPolicy: IfNotPresent
          name: classifier
          resources:
            requests:
              cpu: '0.5'
        terminationGracePeriodSeconds: 1
    graph:
      children: []
      name: classifier
      type: MODEL
    name: example


In [6]:
!kubectl create -f model_with_hpa_v2beta1.yaml -n seldon

seldondeployment.machinelearning.seldon.io/seldon-model created


In [7]:
!kubectl wait sdep/seldon-model \
  --for=condition=ready \
  --timeout=120s \
  -n seldon

seldondeployment.machinelearning.seldon.io/seldon-model condition met


### Create Load

We label some nodes for the loadtester. We attempt the first two as for Kind the first node shown will be the master.

In [8]:
!kubectl label nodes $(kubectl get nodes -o jsonpath='{.items[0].metadata.name}') role=locust

node/kind-control-plane not labeled


In [9]:
!helm install loadtester ../../../helm-charts/seldon-core-loadtesting -n seldon  \
    --set locust.host=http://seldon-model-example:8000 \
    --set oauth.enabled=false \
    --set locust.hatchRate=1 \
    --set locust.clients=1 \
    --set loadtest.sendFeedback=0 \
    --set locust.minWait=0 \
    --set locust.maxWait=0 \
    --set replicaCount=1

NAME: loadtester
LAST DEPLOYED: Thu Dec  4 11:32:40 2025
NAMESPACE: seldon
STATUS: deployed
REVISION: 1
TEST SUITE: None


After a few mins you should see the deployment `my-dep` scaled to 3 deployments

In [10]:
import json
import time


def getNumberPods():
    dp = !kubectl get deployment -n seldon seldon-model-example-0-classifier -o json
    dp = json.loads("".join(dp))
    return dp["status"]["replicas"]


scaled = False
for i in range(60):
    pods = getNumberPods()
    print(pods)
    if pods > 1:
        scaled = True
        break
    time.sleep(5)
assert scaled

1
1
1
1
1
1
1
1
1
3


In [11]:
!kubectl get pods,deployments,hpa -n seldon

NAME                                                     READY   STATUS    RESTARTS   AGE
pod/locust-master-1-92n86                                1/1     Running   0          75s
pod/locust-slave-1-v4cjz                                 1/1     Running   0          75s
pod/seldon-model-example-0-classifier-7b5659c8f9-pcd22   2/2     Running   0          25s
pod/seldon-model-example-0-classifier-7b5659c8f9-vpr2p   2/2     Running   0          115s
pod/seldon-model-example-0-classifier-7b5659c8f9-wpbfm   2/2     Running   0          25s

NAME                                                READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/seldon-model-example-0-classifier   3/3     3            3           116s

NAME                                                                    REFERENCE                                      TARGETS        MINPODS   MAXPODS   REPLICAS   AGE
horizontalpodautoscaler.autoscaling/seldon-model-example-0-classifier   Deployment/seldon-model-example-0-cla

In [12]:
!helm delete loadtester -n seldon

release "loadtester" uninstalled


In [13]:
!kubectl delete -f model_with_hpa_v2beta1.yaml -n seldon

seldondeployment.machinelearning.seldon.io "seldon-model" deleted


## Create model with v2 autoscaler

To create a model with an HorizontalPodAutoscaler there are three steps:


  1. Ensure you have a resource request for the metric you want to scale on if it is a standard metric such as cpu or memory, e.g.:
  
```
          resources:
            requests:
              cpu: '0.5'
     
```
     
  1. Add an v2beta1 HPA Spec referring to this Deployment, e.g.:
  
```
    - hpaSpec:
        maxReplicas: 3
        minReplicas: 1
        metricsv2:
        - resource:
            name: cpu
            target:
              type: Utilization
              averageUtilization: 10
          type: Resource
```

The full SeldonDeployment spec is shown below.

In [15]:
!pygmentize model_with_hpa_v2.yaml

apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: seldon-model
spec:
  name: test-deployment
  predictors:
  - componentSpecs:
    - hpaSpec:
        maxReplicas: 3
        metricsv2:
        - resource:
            name: cpu
            target:
              type: Utilization
              averageUtilization: 10
          type: Resource
        minReplicas: 1
      spec:
        containers:
        - image: seldonio/mock_classifier:1.19.0-dev
          imagePullPolicy: IfNotPresent
          name: classifier
          resources:
            requests:
              cpu: '0.5'
        terminationGracePeriodSeconds: 1
    graph:
      children: []
      name: classifier
      type: MODEL
    name: example


In [16]:
!kubectl create -f model_with_hpa_v2.yaml -n seldon

seldondeployment.machinelearning.seldon.io/seldon-model created


In [17]:
!kubectl wait sdep/seldon-model \
  --for=condition=ready \
  --timeout=120s \
  -n seldon

seldondeployment.machinelearning.seldon.io/seldon-model condition met


### Create Load

We label some nodes for the loadtester. We attempt the first two as for Kind the first node shown will be the master.

In [18]:
!kubectl label nodes $(kubectl get nodes -o jsonpath='{.items[0].metadata.name}') role=locust

node/kind-control-plane not labeled


In [19]:
!helm install loadtester ../../../helm-charts/seldon-core-loadtesting -n seldon  \
    --set locust.host=http://seldon-model-example:8000 \
    --set oauth.enabled=false \
    --set locust.hatchRate=1 \
    --set locust.clients=1 \
    --set loadtest.sendFeedback=0 \
    --set locust.minWait=0 \
    --set locust.maxWait=0 \
    --set replicaCount=1

NAME: loadtester
LAST DEPLOYED: Thu Dec  4 11:35:11 2025
NAMESPACE: seldon
STATUS: deployed
REVISION: 1
TEST SUITE: None


After a few mins you should see the deployment `my-dep` scaled to 3 deployments

In [20]:
import json
import time


def getNumberPods():
    dp = !kubectl get deployment -n seldon seldon-model-example-0-classifier -o json
    dp = json.loads("".join(dp))
    return dp["status"]["replicas"]


scaled = False
for i in range(60):
    pods = getNumberPods()
    print(pods)
    if pods > 1:
        scaled = True
        break
    time.sleep(5)
assert scaled

1
1
1
1
1
1
1
3


In [21]:
!kubectl get pods,deployments,hpa -n seldon

NAME                                                     READY   STATUS    RESTARTS   AGE
pod/locust-master-1-vztf6                                1/1     Running   0          59s
pod/locust-slave-1-98phc                                 1/1     Running   0          59s
pod/seldon-model-example-0-classifier-789cd4b649-47b5j   0/2     Running   0          18s
pod/seldon-model-example-0-classifier-789cd4b649-rkmz2   2/2     Running   0          93s
pod/seldon-model-example-0-classifier-789cd4b649-wrhm9   0/2     Running   0          17s

NAME                                                READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/seldon-model-example-0-classifier   1/3     3            1           93s

NAME                                                                    REFERENCE                                      TARGETS        MINPODS   MAXPODS   REPLICAS   AGE
horizontalpodautoscaler.autoscaling/seldon-model-example-0-classifier   Deployment/seldon-model-example-0-class

In [22]:
!helm delete loadtester -n seldon

release "loadtester" uninstalled


In [23]:
!kubectl delete -f model_with_hpa_v2.yaml -n seldon

seldondeployment.machinelearning.seldon.io "seldon-model" deleted
